### ground truth



Input files:
crsp_clean_26Apr20.csv, rawpv_clean_26Apr2020.csv, new_kpss_clean_11Apr2020.csv

Output files:

groundtruth_kpss_pv_crsp_28Apr20.csv



In [7]:
import pandas as pd
import numpy as np
from cleanco import cleanco
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import  re
import urllib.request as urllib2
import json
import datetime

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
#read crsp clean data
crsp = pd.read_csv("crsp_clean_26Apr20.csv")

In [10]:
crsp.columns

Index(['PERMNO', 'COMNAM', 'PERMCO', 'crsp_year', 'COMNAM2', 'comlow',
       'comlow2', 'last', 'len', 'comlow3', 'comlen', 'last2', 'comlow4'],
      dtype='object')

In [11]:
crsp2 = crsp[['PERMNO', 'COMNAM', 'PERMCO', 'crsp_year', 'comlow4']]

In [12]:
len(crsp2)

333162

In [13]:
#read KPSS raw file as a dataframe
kpss = pd.read_csv('new_kpss_clean_11Apr2020.csv')

In [14]:
len(kpss)

1930089

In [15]:
kpss2 = kpss[kpss['kpss_year']>1975]

In [16]:
kpss_crsp2 = pd.merge(kpss2, crsp2, left_on = ['permno'], right_on = ['PERMNO'], how = "inner")

In [17]:
# check where crsp>=kpssyears
kpss_crsp2['diff'] = (kpss_crsp2['crsp_year']-kpss_crsp2['kpss_year']).abs()

In [18]:
df_min = kpss_crsp2.groupby('patnum').idxmin()

In [19]:
kpss_crsp3 = kpss_crsp2.copy()

In [20]:
kpss_crsp2.loc[df_min['diff'],"yr_match"] = kpss_crsp2.loc[df_min['diff'],"crsp_year"] 

In [21]:
kpss_crsp2['yr_match'].isna().sum() #45165477

45165477

In [22]:
kpss_crsp2 = kpss_crsp2[~kpss_crsp2['yr_match'].isna()]

In [23]:
set(kpss_crsp3.permno.unique()) - set(kpss_crsp2.permno.unique())

{50869.0,
 79195.0,
 80652.0,
 83183.0,
 85654.0,
 85773.0,
 85827.0,
 87296.0,
 88519.0,
 88631.0,
 89223.0,
 89288.0}

In [24]:
pno = [50869.0,79195.0, 80652.0, 83183.0, 85654.0, 85773.0, 85827.0, 87296.0,88519.0, 88631.0, 89223.0, 89288.0]
cmn = kpss_crsp3[(kpss_crsp3['PERMNO'].isin(pno))]

In [25]:
cmn.patnum.nunique()

23

In [26]:
df_min2 = cmn.groupby('patnum').idxmin()

In [27]:
cmn.loc[df_min2['diff'],"yr_match"] = cmn.loc[df_min2['diff'],"crsp_year"] 

In [28]:
cmn = cmn[~cmn['yr_match'].isna()]

In [29]:
cmn.PERMNO.nunique()

12

In [30]:
kpss_crsp2 = kpss_crsp2.append(cmn)

In [31]:
kc = kpss_crsp2[['patnum', 'PERMNO', 'PERMCO','COMNAM','comlow4']]

In [32]:
kc = kc.drop_duplicates(keep = 'first')

In [33]:
kc.to_csv("kc_26Apr20.csv", index = False) 

In [36]:
# kc = pd.read_csv("kc_23Apr20.csv")

In [37]:
############# map with patent view ########

In [38]:
pv = pd.read_csv("rawpv_clean_26Apr2020.csv")

In [39]:
pv.columns

Index(['patent_id', 'assignee_id', 'type', 'organization', 'sequence',
       'bractext', 'organization2', 'organization3', 'bractext1', 'orglow',
       'orglow2', 'orglow3', 'orglow4', 'orglow5', 'orglow6', 'last',
       'orglow7', 'orglow8', 'orglow9'],
      dtype='object')

In [40]:
kc['patnum'] = kc['patnum'].astype('str')

In [41]:
kcp = pd.merge(pv, kc, left_on = ['patent_id'],right_on = ['patnum'], how = "outer")

In [42]:
kcp2 = kcp[~kcp['patnum'].isna()]
kcp2 = kcp2[~kcp2['patent_id'].isna()]

In [43]:
kcp2.columns

Index(['patent_id', 'assignee_id', 'type', 'organization', 'sequence',
       'bractext', 'organization2', 'organization3', 'bractext1', 'orglow',
       'orglow2', 'orglow3', 'orglow4', 'orglow5', 'orglow6', 'last',
       'orglow7', 'orglow8', 'orglow9', 'patnum', 'PERMNO', 'PERMCO', 'COMNAM',
       'comlow4'],
      dtype='object')

In [44]:
gt = kcp2[['patent_id',  'organization',
       'orglow9', 'PERMNO', 'PERMCO', 'COMNAM',
       'comlow4','assignee_id', 'type', 'sequence']]

In [45]:
gt.nunique()

patent_id       1392370
organization      41610
orglow9           21329
PERMNO             6923
PERMCO             6794
COMNAM             7797
comlow4            7535
assignee_id       20822
type                  2
sequence             14
dtype: int64

In [46]:
gt['orglow9'] = gt['orglow9'].str.replace("united states","us")
gt['comlow4'] = gt['comlow4'].str.replace("united states","us")

gt['comlow4'] = gt['comlow4'].str.replace(" entnmt"," entertainment")
gt['comlow4'] = gt['comlow4'].str.replace(" frag"," fragrances")



In [47]:
## direct match ## without cleaning

In [48]:
gt['orglow_dm'] = gt['organization'].apply(lambda x: x.lower())
gt['comlow_dm'] = gt['COMNAM'].apply(lambda x: x.lower())

In [49]:
gt['dm'] = gt['orglow_dm'] == gt['comlow_dm']

In [50]:
gt[gt['dm']==True][['PERMNO','organization','COMNAM','patent_id']].nunique() 
# PERMNO 281, organization 355, COMNAM 283,patent_id 15307

PERMNO            281
organization      355
COMNAM            283
patent_id       15307
dtype: int64

In [51]:
####direct match before cleaning after removing spaces

In [52]:
gt['orglow_dmns'] = gt['orglow_dm'].apply(lambda x: x.replace(" ",""))
gt['comlow_dmns'] = gt['comlow_dm'].apply(lambda x: x.replace(" ",""))

In [53]:
gt['dm_ns'] = gt['orglow_dmns'] == gt['comlow_dmns']

In [54]:
gt[gt['dm_ns']==True][['PERMNO','organization','COMNAM','patent_id']].nunique() 
# PERMNO 347, organization 531, COMNAM 351,patent_id 26565

PERMNO            347
organization      531
COMNAM            351
patent_id       26565
dtype: int64

In [55]:
gt[(gt['dm_ns']==True)&(gt['dm']!=True)][['orglow_dmns','organization','COMNAM','comlow_dmns']].sort_values('COMNAM')

,orglow_dmns,organization,COMNAM,comlow_dmns
2596882,abbab,ABB AB,A B B AB,abbab
5301553,abbab,ABB AB,A B B AB,abbab
5775543,abbab,ABB AB,A B B AB,abbab
5842453,abbab,ABB AB,A B B AB,abbab
5772717,abbab,ABB AB,A B B AB,abbab
...,...,...,...,...
176922,xeikonnv,Xeikon NV,XEIKON N V,xeikonnv
4983212,xeikonnv,Xeikon NV,XEIKON N V,xeikonnv
5744479,xeikonnv,Xeikon NV,XEIKON N V,xeikonnv
5628384,xeikonnv,Xeikon NV,XEIKON N V,xeikonnv


In [56]:
########direct match after cleaning #######

In [57]:
gt['clean_dm'] = gt['orglow9'] == gt['comlow4']

In [58]:
gt[gt['clean_dm']==True][['PERMNO','organization','COMNAM','patent_id']].nunique()
# PERMNO 5913; organization 16780, COMNAM 6335; patent_id =1006778 

PERMNO             5913
organization      16780
COMNAM             6336
patent_id       1006778
dtype: int64

In [59]:
gt[gt['clean_dm']==True][['orglow9','organization','COMNAM','comlow4']]

,orglow9,organization,COMNAM,comlow4
1,xerox,Xerox Corporation,XEROX CORP,xerox
5,cypress semiconductor,Cypress Semiconductor Corp.,CYPRESS SEMICONDUCTOR CORP,cypress semiconductor
6,motorola,"Motorola, Inc.",MOTOROLA INC,motorola
15,sony,Sony Corporation,SONY CORP,sony
27,raytheon,Raytheon Company,RAYTHEON CO,raytheon
...,...,...,...,...
6258150,bristol myers squibb,Bristol-Myers Squibb Company,BRISTOL MYERS SQUIBB CO,bristol myers squibb
6258153,att,AT&T Corp.,A T & T CORP,att
6258164,e systems,"E-Systems, Inc.",E SYSTEMS INC,e systems
6258166,canon,Canon Kabushiki Kaisha,CANON INC,canon


In [60]:
###########direct match after cleaning after removing spaces################

In [61]:
gt['orglow9_ns'] = gt['orglow9'].apply(lambda x: x.replace(" ",""))
gt['comlow4_ns'] = gt['comlow4'].apply(lambda x: x.replace(" ",""))

In [62]:
gt['cleandm_ns'] = gt['orglow9_ns'] == gt['comlow4_ns']

In [63]:
gt[gt['cleandm_ns']==True][['PERMNO','organization','COMNAM','patent_id']].nunique()
# PERMNO 6052; organization 17603, COMNAM 6498, patent_id = 1034762

PERMNO             6052
organization      17603
COMNAM             6498
patent_id       1034762
dtype: int64

In [ ]:
######################### mathing remaining ##########################################################

In [64]:
tm = gt[gt['cleandm_ns']==False]

In [65]:
tm['cleandm_ns'].value_counts()

False    388409
Name: cleandm_ns, dtype: int64

In [66]:
#datafrmae without patents

temp = tm[['assignee_id', 'PERMNO', 'COMNAM',
       'organization',  'orglow9', 'comlow4']]

In [67]:
temp = temp.drop_duplicates(keep = 'first')

In [68]:
temp.columns

Index(['assignee_id', 'PERMNO', 'COMNAM', 'organization', 'orglow9',
       'comlow4'],
      dtype='object')

In [69]:
len(temp)

29912

In [70]:
temp['fuzz_ratio'] = temp.apply(lambda x: fuzz.ratio(x.orglow9, x.comlow4), axis = 1)
temp['fuzz_token'] = temp.apply(lambda x: fuzz.token_set_ratio(x.orglow9, x.comlow4), axis = 1)
temp['fuzz_fpr'] = temp.apply(lambda x: fuzz.partial_ratio(x.orglow9, x.comlow4), axis = 1)

In [71]:
temp[(temp['fuzz_token']<90)&(temp['fuzz_ratio']<90)&(temp['fuzz_fpr']>=90)
     ]

,assignee_id,PERMNO,COMNAM,organization,orglow9,comlow4,fuzz_ratio,fuzz_token,fuzz_fpr
195,org_rjTdv98kllFyITzDnJQj,58640.0,NORTEL NETWORKS CORP NEW,Nortel Networks Limited,northern telecommunication,northern telecommunications networks,84,84,100
209,org_X2oSuoCvOY1jbTB3PTGS,88487.0,KONINKLIJKE PHILIPS ELEC N V,Koninklijke Philips Electronics N.V.,koninklijke philips electronics,koninklijke philips elec,87,88,100
988,org_rjTdv98kllFyITzDnJQj,58640.0,NORTEL NETWORKS CORP,Nortel Networks Corporation,northern telecommunication,northern telecommunications networks,84,84,100
2622,org_rjTdv98kllFyITzDnJQj,58640.0,NORTEL NETWORKS CORP NEW,Nortel Networks Ltd,northern telecommunication,northern telecommunications networks,84,84,100
2830,org_Bsjz1zmq7eIkCMBmfufu,11850.0,EXXON MOBIL CORP,ExxonMobil Chemical Patents Inc.,exxonmobil chemical patents,exxon mobil,53,53,91
...,...,...,...,...,...,...,...,...,...
6187063,org_wsUVk0BaMWye2y1aZRH4,52919.0,MERRILL LYNCH & CO INC,"Merril Lynch, Pierce, Fenner & Smith Inc.",merril lynch pierce fenner smith,merrill lynch,53,56,92
6202488,org_oI7MQtpApv1WiTrI3EpV,85010.0,GENESYS TELECOM LAB INC,"Gewesys Telecommunications Laboratories, Inc.",gewesys telecommunications laboratories,genesys telecommunication lab,82,82,93
6224454,org_QGbE0Z7nZthhGjyweJA3,82643.0,LEXMARK INTERNATIONAL GROUP INC,"Lexmark Internatonal, Inc.",lexmark internatonal,lexmark international group,85,85,95
6236894,org_Bhxt9OMrTI8tGEuPPcbu,28417.0,MILES LABS INC,Miles Laboratories Inc.,miles laboratories,miles labs,71,71,90


In [72]:
temp['orglow_first'] = temp['orglow9'].apply(lambda x: x[0])
temp['comlow_first'] = temp['comlow4'].apply(lambda x: x[0])

In [73]:
temp['same'] = temp['orglow_first'] == temp['comlow_first']

In [74]:
hifq = ['north american','works','motor','corportion', 'worldwide', 'deutschland',
    'industries','industry', 'of', 'systems', 'technology', 'electric', 'electronics', 
         'group','groups','manufacturing', 'pharmaceuticals', 'products','pharmaceutical'
        'technologies','company', 'de','international',
       'the','system','services','industrial','electronic','holdings',
       'america','japan','service','for','gesellschaft','communication','communications',
       'global','kabushiki','la','network', 'networks','us','tokyo','und','new england',
        'new holland','ltd','american', 'corp', 'corporation','development','enterprises',
         'holding','industry','lm','national','product','technologies','tech',
 'united states','united']

In [75]:
temp['comlow_new'] = temp['comlow4']
temp['orglow_new'] = temp['orglow9']

In [76]:
for i in hifq:
    j = " "+i+" "
    y = " "+i+"$"
    z = "^" + i +" "
 
    temp.loc[temp['comlow4'].str.contains(i),'comlow_new'] = temp[temp['comlow4'].str.contains(i)]['comlow_new'].str.replace(j," ")
    temp.loc[temp['orglow9'].str.contains(i),'orglow_new'] = temp[temp['orglow9'].str.contains(i)]['orglow_new'].str.replace(j," ")
    temp.loc[temp['comlow4'].str.endswith(i),'comlow_new'] = temp[temp['comlow4'].str.endswith(i)]['comlow_new'].str.replace(y,"")
    temp.loc[temp['orglow9'].str.endswith(i),'orglow_new'] = temp[temp['orglow9'].str.endswith(i)]['orglow_new'].str.replace(y,"")
    temp.loc[temp['comlow4'].str.startswith(i),'comlow_new'] = temp[temp['comlow4'].str.startswith(i)]['comlow_new'].str.replace(z,"")
    temp.loc[temp['orglow9'].str.startswith(i),'orglow_new'] = temp[temp['orglow9'].str.startswith(i)]['orglow_new'].str.replace(z,"")
    

In [77]:
lst = ['labs', 'lab','laboratories','laboratory','intellectual property']
for i in lst:
    y = " "+i+"$"
    temp.loc[temp['comlow_new'].str.endswith(i),'comlow_new'] = temp[temp['comlow_new'].str.endswith(i)]['comlow_new'].str.replace(y,"")
    temp.loc[temp['orglow_new'].str.endswith(i),'orglow_new'] = temp[temp['orglow_new'].str.endswith(i)]['orglow_new'].str.replace(y,"")

In [78]:
lst = ['corp', 'corporation','technology','technologies']

for i in lst:
    y = i+"$"
    temp.loc[temp['comlow_new'].str.endswith(i),'comlow_new'] = temp[temp['comlow_new'].str.endswith(i)]['comlow_new'].str.replace(y,"")
    temp.loc[temp['orglow_new'].str.endswith(i),'orglow_new'] = temp[temp['orglow_new'].str.endswith(i)]['orglow_new'].str.replace(y,"")

In [79]:
temp['comlow_new'] = temp['comlow_new'].apply(lambda x: x.strip())
temp['orglow_new'] = temp['orglow_new'].apply(lambda x: x.strip())

In [80]:
temp.loc[temp['comlow_new']=="","comlow_new"] = temp.loc[temp['comlow_new']=="","comlow4"]
temp.loc[temp['orglow_new']=="","orglow_new"] = temp.loc[temp['orglow_new']=="","orglow9"]

In [81]:
# short forms of companies after low
temp['orglow_short'] = temp['orglow9'].apply(lambda x: ''.join(word[0] for word in x.split()))
temp['comlow_short'] = temp['comlow4'].apply(lambda x: ''.join(word[0] for word in x.split()))

In [82]:
# short forms of companies before cleaning
temp['org_short'] = temp['organization'].apply(lambda x: (''.join(word[0] for word in x.split())).lower())
temp['com_short'] = temp['COMNAM'].apply(lambda x: (''.join(word[0] for word in x.split())).lower())

In [83]:
##fuzz matching with new

temp['new_fr'] = temp.apply(lambda x: fuzz.ratio(x.orglow_new, x.comlow_new), axis = 1)
temp['new_ft'] = temp.apply(lambda x: fuzz.token_set_ratio(x.orglow_new, x.comlow_new), axis = 1)
temp['new_fpr'] = temp.apply(lambda x: fuzz.partial_ratio(x.orglow_new, x.comlow_new), axis = 1)

In [84]:
temp['orgnew_first'] = temp['orglow_new'].apply(lambda x: x[0])
temp['comnew_first'] = temp['comlow_new'].apply(lambda x: x[0])
temp['same_new'] = temp['orgnew_first']==temp['comnew_first']

In [85]:
temp['ofuzz_shortlow'] = temp.apply(lambda x: fuzz.token_set_ratio(x.orglow_short, x.comlow4), axis = 1)
temp['cfuzz_shortlow'] = temp.apply(lambda x: fuzz.token_set_ratio(x.comlow_short, x.orglow9), axis = 1)

temp['ofuzz_short'] = temp.apply(lambda x: fuzz.token_set_ratio(x.org_short, x.COMNAM), axis = 1)
temp['cfuzz_short'] = temp.apply(lambda x: fuzz.token_set_ratio(x.com_short, x.organization), axis = 1)



In [86]:
temp.loc[((temp['new_fr']==90)|(temp['new_ft']==90)|(temp['new_fpr']==90))&
     (temp['same_new']==True)][['new_fr','new_ft','new_fpr','fuzz_ratio','fuzz_token','fuzz_fpr',
                                'COMNAM','organization']].tail(80).sort_values('fuzz_token')

,new_fr,new_ft,new_fpr,fuzz_ratio,fuzz_token,fuzz_fpr,COMNAM,organization
4270227,71,43,90,49,29,90,SMITHKLINE CORP,Smith Kline & French Laboratories Limited
6124558,71,43,90,71,43,90,SMITHKLINE CORP,Smith Kline & French (Proprietary) Limited
3144465,90,90,90,55,55,90,KOLLMORGEN CORP,Kollmorgan Technologies Corporation
3697382,62,62,90,62,62,90,BORGWARNER INC,"Borg-Warner Automotive, Inc."
3868248,62,62,90,73,65,80,MATSUSHITA ELECTRIC INDL LTD,Marsushita Electric Works. Ltd.
...,...,...,...,...,...,...,...,...
4780271,68,100,90,62,100,93,AMERICAN TELEPHONE & TELEG CO,"American Telephone & Telegraph Company, AT&T I..."
4766756,86,100,90,67,100,93,AMERICAN TELEPHONE & TELEG CO,"American Telephone and Telegraph Company, AT&T..."
4708591,78,100,90,64,100,93,AMERICAN TELEPHONE & TELEG CO,"American Telephone Telegraph Company, AT&T Bel..."
5651277,68,100,90,68,100,93,AMERICAN TELEPHONE & TELEG CO,"American Telephone and Telegraph, AT&T Informa..."


In [87]:
temp.loc[((temp['new_fr']>=90)|(temp['new_ft']>=90)|(temp['new_fpr']>=90)), "match"] = True

In [88]:
len(temp[temp['match'].isna()])

17248

In [89]:
#22509.0 : false; 5160121,4650873,4450088, 4356894: false

In [90]:
temp.loc[(temp['match'].isna())&((temp['fuzz_ratio']>=95)| 
                             (temp['fuzz_token']>=95)), "match"] = True

In [91]:
temp.loc[(temp['match'].isna())&((temp['fuzz_ratio']>=94)&
                             (temp['fuzz_token']>=94)),"match"] = True

In [92]:
len(temp[temp['match'].isna()])

17167

In [93]:
temp.columns

Index(['assignee_id', 'PERMNO', 'COMNAM', 'organization', 'orglow9', 'comlow4',
       'fuzz_ratio', 'fuzz_token', 'fuzz_fpr', 'orglow_first', 'comlow_first',
       'same', 'comlow_new', 'orglow_new', 'orglow_short', 'comlow_short',
       'org_short', 'com_short', 'new_fr', 'new_ft', 'new_fpr', 'orgnew_first',
       'comnew_first', 'same_new', 'ofuzz_shortlow', 'cfuzz_shortlow',
       'ofuzz_short', 'cfuzz_short', 'match'],
      dtype='object')

In [94]:
temp['oshrtlow_len'] = temp['orglow_short'].apply(lambda x: len(x))
temp['cshrtlow_len'] = temp['comlow_short'].apply(lambda x: len(x))
temp['oshrt_len'] = temp['org_short'].apply(lambda x: len(x))
temp['cshrt_len'] = temp['com_short'].apply(lambda x: len(x))
temp['org_len'] = temp['orglow9'].apply(lambda x: len(x))
temp['com_len'] = temp['comlow4'].apply(lambda x: len(x))


In [95]:
#temp.loc[(temp['cfuzz_shortlow']==100)&(temp['cshrtlow_len']==2)&(temp['match'].isna())]

# temp.loc[(temp['ofuzz_shortlow']==100)&(temp['oshrtlow_len']==3)&(temp['match'].isna())]



In [97]:
temp.loc[(temp['cfuzz_shortlow']==100)&(temp['cshrtlow_len']>=2)&(temp['match'].isna()), "match"] = True

temp.loc[(temp['ofuzz_shortlow']==100)&(temp['oshrtlow_len']>=2)&(temp['match'].isna()),"match"] = True

In [98]:
# temp.loc[(temp['cfuzz_short']==100)&(temp['com_len']>=5)&
#          (temp['match'].isna())&(temp['cshrt_len']==3)][['cshrt_len','comlow4','orglow9']]



In [99]:
len(temp[temp['match'].isna()])

16798

In [100]:
temp2 = pd.read_csv("fm.csv")

In [101]:
temp3 = temp2[~(temp2['match'].isna())& (~(temp2['cleandm_ns']==True))]

In [102]:
del temp3['patent_id']

In [103]:
temp3.columns

Index(['organization', 'orglow9', 'PERMNO', 'PERMCO', 'COMNAM', 'comlow4',
       'assignee_id_x', 'type', 'sequence', 'orglow_dm', 'comlow_dm', 'dm',
       'orglow_dmns', 'comlow_dmns', 'dm_ns', 'clean_dm', 'orglow9_ns',
       'comlow4_ns', 'cleandm_ns', 'assignee_id_y', 'match'],
      dtype='object')

In [104]:
temp4 = temp3[['organization', 'orglow9', 'PERMNO', 'PERMCO', 'COMNAM', 'comlow4','match']]

In [105]:
temp4 = temp4.drop_duplicates(keep = 'first')

In [106]:
temp4.match.value_counts()

TRUE                           11266
tomatch                         1428
shoud match                      742
FALSE                            413
should match on lang change       78
short form                        54
Name: match, dtype: int64

In [107]:
temp4.rename(columns = {"match":"temp_match"}, inplace = True)

In [108]:
temp4.columns

Index(['organization', 'orglow9', 'PERMNO', 'PERMCO', 'COMNAM', 'comlow4',
       'temp_match'],
      dtype='object')

In [109]:
temp.columns

Index(['assignee_id', 'PERMNO', 'COMNAM', 'organization', 'orglow9', 'comlow4',
       'fuzz_ratio', 'fuzz_token', 'fuzz_fpr', 'orglow_first', 'comlow_first',
       'same', 'comlow_new', 'orglow_new', 'orglow_short', 'comlow_short',
       'org_short', 'com_short', 'new_fr', 'new_ft', 'new_fpr', 'orgnew_first',
       'comnew_first', 'same_new', 'ofuzz_shortlow', 'cfuzz_shortlow',
       'ofuzz_short', 'cfuzz_short', 'match', 'oshrtlow_len', 'cshrtlow_len',
       'oshrt_len', 'cshrt_len', 'org_len', 'com_len'],
      dtype='object')

In [110]:
temp.match.value_counts()

True    13114
Name: match, dtype: int64

In [111]:
tt = pd.merge(temp4, temp, left_on = ['organization', 'orglow9', 'PERMNO', 'COMNAM', 'comlow4'],
                right_on = ['organization', 'orglow9', 'PERMNO', 'COMNAM', 'comlow4'],
                how = "right")

In [112]:
tt.columns.sort_values()

Index(['COMNAM', 'PERMCO', 'PERMNO', 'assignee_id', 'cfuzz_short',
       'cfuzz_shortlow', 'com_len', 'com_short', 'comlow4', 'comlow_first',
       'comlow_new', 'comlow_short', 'comnew_first', 'cshrt_len',
       'cshrtlow_len', 'fuzz_fpr', 'fuzz_ratio', 'fuzz_token', 'match',
       'new_fpr', 'new_fr', 'new_ft', 'ofuzz_short', 'ofuzz_shortlow',
       'org_len', 'org_short', 'organization', 'orglow9', 'orglow_first',
       'orglow_new', 'orglow_short', 'orgnew_first', 'oshrt_len',
       'oshrtlow_len', 'same', 'same_new', 'temp_match'],
      dtype='object')

In [113]:
len(tt)

29912

In [114]:
tt.loc[tt['match']==True,'temp_match'] = "TRUE"

In [115]:
tt.temp_match.value_counts()

TRUE                           13132
shoud match                      464
FALSE                            410
should match on lang change       78
short form                        49
tomatch                            1
Name: temp_match, dtype: int64

In [116]:
tt.loc[tt['temp_match']=="tomatch","temp_match"] = "TRUE"

In [117]:
tt.temp_match.value_counts()

TRUE                           13133
shoud match                      464
FALSE                            410
should match on lang change       78
short form                        49
Name: temp_match, dtype: int64

In [118]:
pv[pv['assignee_id']=="org_wRmTDOlFQn70Uye4jiNj"]


,patent_id,assignee_id,type,organization,sequence,bractext,organization2,organization3,bractext1,orglow,orglow2,orglow3,orglow4,orglow5,orglow6,last,orglow7,orglow8,orglow9
531502,4428153,org_wRmTDOlFQn70Uye4jiNj,2,Atlanta Richfield Company,0,NaN,Atlanta Richfield Company,Atlanta Richfield Company,NaN,atlanta richfield company,atlanta richfield company,atlanta richfield,atlanta richfield,atlanta richfield,atlanta richfield,richfield,atlanta richfield,atlanta richfield,atlanta richfield


In [120]:
tt[tt['temp_match']=="short form"][['organization','COMNAM','comlow4','orglow9','assignee_id']].sort_values(['organization'])

,organization,COMNAM,comlow4,orglow9,assignee_id
13696,"501 Aeroflex Laboratories, Incorporated",A R X INC,arx,501 aeroflex laboratories,org_iEg6HKZ5tpLonvMC7JOs
12547,"ACRO Chemical Technology, L.P.",ATLANTIC RICHFIELD CO,arco,acro chemical technology,org_C42kCVDQIlryXIV3q4QK
3073,Aeroflex International Incorporated,A R X INC,arx,aeroflex,org_f24r0meHZd1jU5ct5xyi
2929,"Aircraft Mechanics, Inc.",A M I INDUSTRIES INC CO,ami industries,aircraft mechanics,org_yUCp6mJNoZ74PUM3fb4D
11622,American Airlines,A M R CORP DEL,amr,american airlines,org_30xNJAOapGVsZ4TEMmTs
9185,American Airlines Incorporated,A M R CORP DEL,amr,american airlines,org_30xNJAOapGVsZ4TEMmTs
1351,"American Airlines, Inc.",A M R CORP DEL,amr,american airlines,org_30xNJAOapGVsZ4TEMmTs
12747,"American Airlines, Incorporated",A M R CORP DEL,amr,american airlines,org_30xNJAOapGVsZ4TEMmTs
3569,Atlanta Richfield Company,ATLANTIC RICHFIELD CO,arco,atlanta richfield,org_wRmTDOlFQn70Uye4jiNj
6915,Atlantic Refining Company,ATLANTIC RICHFIELD CO,arco,atlantic refining,org_RylM1LndC1cID4JOEAVz


In [121]:
tt1.loc[tt1.temp_match.isin(["shoud match", "TRUE","should match on lang change","short form"]),"nmatch"] = True

In [122]:
tt.loc[tt['nmatch']!=True,"nmatch"] = False

In [123]:
tt.nmatch.value_counts()

False    16192
True     13720
Name: nmatch, dtype: int64

In [124]:
#temp2 = pd.read_csv("temptoday.csv")

In [224]:
fm = pd.merge(gt, tt1 ,left_on = ['PERMNO', 'COMNAM', 'organization', 'orglow9', 'comlow4'], 
              right_on = ['PERMNO', 'COMNAM', 'organization', 'orglow9', 'comlow4'], 
              
              how = "left")

In [225]:
fm.columns.sort_values()

Index(['COMNAM', 'PERMCO', 'PERMNO', 'assignee_id', 'clean_dm', 'cleandm_ns',
       'com_short', 'comlow4', 'comlow4_ns', 'comlow_dm', 'comlow_dmns',
       'comlow_new', 'comlow_short', 'dm', 'dm_ns', 'fuzz_fpr', 'fuzz_ratio',
       'fuzz_token', 'match', 'new_fpr', 'new_fr', 'new_ft', 'nmatch',
       'org_short', 'organization', 'orglow9', 'orglow9_ns', 'orglow_dm',
       'orglow_dmns', 'orglow_new', 'orglow_short', 'patent_id', 'sequence',
       'temp_match', 'type'],
      dtype='object')

In [226]:
fm.loc[(fm['cleandm_ns']==True) |(fm['nmatch'] ==True),"finalmatch"] = True

In [128]:
####fm.to_csv("fm.csv", index = False)

In [227]:
fm.finalmatch.value_counts()

True    1770192
Name: finalmatch, dtype: int64

In [228]:
matched = list(fm[(fm['cleandm_ns']==True) |(fm['nmatch'] ==True)]['PERMNO'].unique()) #6728

In [229]:
len(matched) #6728

6728

In [230]:
initial = list(gt['PERMNO'].unique())

In [231]:
diff = list(set(initial)-set(matched))

In [232]:
diff.sort()

In [233]:
diff

[10092.0,
 10108.0,
 10506.0,
 10522.0,
 10779.0,
 11156.0,
 11198.0,
 11216.0,
 11553.0,
 11781.0,
 11891.0,
 11950.0,
 12131.0,
 12749.0,
 12926.0,
 14824.0,
 14859.0,
 16548.0,
 17161.0,
 17778.0,
 17814.0,
 18817.0,
 20183.0,
 20205.0,
 20917.0,
 21566.0,
 22461.0,
 22605.0,
 23246.0,
 23501.0,
 25056.0,
 26382.0,
 26593.0,
 27027.0,
 27924.0,
 28118.0,
 28804.0,
 29496.0,
 30680.0,
 31042.0,
 31456.0,
 31472.0,
 31595.0,
 32628.0,
 32716.0,
 33145.0,
 34067.0,
 36003.0,
 36687.0,
 36927.0,
 37698.0,
 37815.0,
 39669.0,
 40272.0,
 40708.0,
 41188.0,
 42067.0,
 42075.0,
 42569.0,
 42972.0,
 43107.0,
 43211.0,
 43617.0,
 43887.0,
 43967.0,
 44265.0,
 45014.0,
 45103.0,
 45524.0,
 45751.0,
 46199.0,
 46499.0,
 47204.0,
 47395.0,
 48397.0,
 48506.0,
 48565.0,
 49787.0,
 50980.0,
 51254.0,
 51721.0,
 51829.0,
 51933.0,
 52492.0,
 53620.0,
 54332.0,
 54340.0,
 54528.0,
 55546.0,
 55626.0,
 55853.0,
 57286.0,
 57402.0,
 59010.0,
 59379.0,
 59504.0,
 60354.0,
 60417.0,
 60660.0,
 61292.0,


In [234]:
fm.nmatch.value_counts()

True     734857
False    184805
Name: nmatch, dtype: int64

In [235]:
fm[fm['nmatch'] == False]['PERMNO'].nunique() ##6728; 195

1903

In [236]:
final_gt = fm[(fm['nmatch'] ==True)|(fm['cleandm_ns']==True)]

In [237]:
final_gt.columns

Index(['patent_id', 'organization', 'orglow9', 'PERMNO', 'PERMCO', 'COMNAM',
       'comlow4', 'assignee_id', 'type', 'sequence', 'orglow_dm', 'comlow_dm',
       'dm', 'orglow_dmns', 'comlow_dmns', 'dm_ns', 'clean_dm', 'orglow9_ns',
       'comlow4_ns', 'cleandm_ns', 'temp_match', 'fuzz_ratio', 'fuzz_token',
       'fuzz_fpr', 'comlow_new', 'orglow_new', 'orglow_short', 'comlow_short',
       'org_short', 'com_short', 'new_fr', 'new_ft', 'new_fpr', 'match',
       'nmatch', 'finalmatch'],
      dtype='object')

In [141]:
final_gt.rename(columns = {"PERMCO_x":"PERMCO","assignee_id_x":"assignee_id" }, inplace = True)

In [142]:
del final_gt['PERMCO_y'], final_gt['assignee_id_y']

In [144]:
final_gt.finalmatch.value_counts()

True    1770196
Name: finalmatch, dtype: int64

In [145]:
len(final_gt)

1770196

In [146]:
final_gt[(final_gt['nmatch']==True) | (final_gt['cleandm_ns']==True)]

,patent_id,organization,orglow9,PERMNO,PERMCO,COMNAM,comlow4,assignee_id,type,sequence,...,cfuzz_short,match,oshrtlow_len,cshrtlow_len,oshrt_len,cshrt_len,org_len,com_len,nmatch,finalmatch
0,5204210,Xerox Corporation,xerox,27983.0,21945.0,XEROX CORP,xerox,org_UrbE3xev7LUsnuvRjbep,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,6584517,Cypress Semiconductor Corp.,cypress semiconductor,10302.0,8222.0,CYPRESS SEMICONDUCTOR CORP,cypress semiconductor,org_sj7olrHxASyJDNVGczBe,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,6970439,"Motorola, Inc.",motorola,22779.0,21230.0,MOTOROLA INC,motorola,org_E6m1uPIsSbg4FlEl54lY,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,6940219,Sony Electronics Inc.,sony electronics,51131.0,21633.0,SONY CORP,sony,org_bjjMpz8jmP43oeCb4YxD,2.0,1.0,...,18.0,True,2.0,1.0,3.0,2.0,16.0,4.0,True,True
5,6940219,Sony Corporation,sony,51131.0,21633.0,SONY CORP,sony,org_sCgv9ie5243t4B3daUSo,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954996,5513247,AT&T Corp.,att,10401.0,20103.0,A T & T CORP,att,org_tBdNUkbblSIviAl1Vcj9,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1954997,4388644,"E-Systems, Inc.",e systems,43342.0,20620.0,E SYSTEMS INC,e systems,org_dc4S7Fb8htIdAGbbNs2h,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1954998,6372818,Canon Kabushiki Kaisha,canon,21152.0,748.0,CANON INC,canon,org_7fE5f5nnY6dbOc3vSaXb,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1954999,5519728,National Semiconductor Corporation,national semiconductor,51377.0,21261.0,NATIONAL SEMICONDUCTOR CORP,national semiconductor,org_S3xLWAv5Md0Losa4TOqB,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [147]:
final_gt = final_gt.drop_duplicates(keep = 'first')

In [148]:
len(final_gt)

1288302

In [149]:
final_gt.nunique()

patent_id         1278759
organization        30328
orglow9             13774
PERMNO               6728
PERMCO               6610
COMNAM               7371
comlow4              7130
assignee_id         13490
type                    2
sequence               11
orglow_dm           27810
comlow_dm            7371
dm                      2
orglow_dmns         23647
comlow_dmns          7367
dm_ns                   2
clean_dm                2
orglow9_ns          13481
comlow4_ns           7122
cleandm_ns              2
temp_match              4
fuzz_ratio             89
fuzz_token             93
fuzz_fpr               76
orglow_first           34
comlow_first           29
same                    2
comlow_new           2303
orglow_new           7220
orglow_short         2547
comlow_short          782
org_short            4410
com_short            1409
new_fr                 93
new_ft                 90
new_fpr                70
orgnew_first           34
comnew_first           29
same_new    

In [150]:
final_gt['nmatch'].value_counts()

True    252967
Name: nmatch, dtype: int64

In [151]:
final_gt.columns

Index(['patent_id', 'organization', 'orglow9', 'PERMNO', 'PERMCO', 'COMNAM',
       'comlow4', 'assignee_id', 'type', 'sequence', 'orglow_dm', 'comlow_dm',
       'dm', 'orglow_dmns', 'comlow_dmns', 'dm_ns', 'clean_dm', 'orglow9_ns',
       'comlow4_ns', 'cleandm_ns', 'temp_match', 'fuzz_ratio', 'fuzz_token',
       'fuzz_fpr', 'orglow_first', 'comlow_first', 'same', 'comlow_new',
       'orglow_new', 'orglow_short', 'comlow_short', 'org_short', 'com_short',
       'new_fr', 'new_ft', 'new_fpr', 'orgnew_first', 'comnew_first',
       'same_new', 'ofuzz_shortlow', 'cfuzz_shortlow', 'ofuzz_short',
       'cfuzz_short', 'match', 'oshrtlow_len', 'cshrtlow_len', 'oshrt_len',
       'cshrt_len', 'org_len', 'com_len', 'nmatch', 'finalmatch'],
      dtype='object')

In [160]:
final_gtm.columns.sort_values()

Index(['COMNAM', 'PERMCO', 'PERMNO', 'assignee_id', 'cfuzz_short',
       'cfuzz_shortlow', 'clean_dm', 'cleandm_ns', 'com_len', 'com_short',
       'comlow4', 'comlow4_ns', 'comlow_dm', 'comlow_dmns', 'comlow_first',
       'comlow_new', 'comlow_short', 'comnew_first', 'cshrt_len',
       'cshrtlow_len', 'dm', 'dm_ns', 'finalmatch', 'fuzz_fpr', 'fuzz_ratio',
       'fuzz_token', 'match', 'new_fpr', 'new_fr', 'new_ft', 'nmatch',
       'ofuzz_short', 'ofuzz_shortlow', 'org_len', 'org_short', 'organization',
       'orglow9', 'orglow9_ns', 'orglow_dm', 'orglow_dmns', 'orglow_first',
       'orglow_new', 'orglow_short', 'orgnew_first', 'oshrt_len',
       'oshrtlow_len', 'patent_id', 'same', 'same_new', 'sequence',
       'shortform', 'temp_match', 'type'],
      dtype='object')

In [252]:
final_gtm[~final_gtm['shortform'].isna()][['organization','COMNAM','comlow4',
                                           'orglow9','assignee_id']].head(20)

,organization,COMNAM,comlow4,orglow9,assignee_id
2779,"PCC Airfoils, Inc.",PRECISION CASTPARTS CORP,precision castparts,pcc airfoils,org_Pln2mHPqOo8ivj1rdmfC
3902,"The International Nickel Company, Inc.",INCO LTD,inco,international nickel,org_3O12UKd8pdv6Gj2KNctq
4029,SLM Canada Inc.,STUDENT LOAN MARKETING ASSN,student loan marketing,slm canada,org_e7fzqKTbstVEvLCusYFz
4295,International Telephone and Telegraph Corporation,I T T CORP,itt,international telephone telegraph,org_U2TRh9WomWWWDaH5ybhG
5148,Union Oil Company of California,UNOCAL CORP,unocal,union oil company of california,org_COgAe5MtbLYwQVAiE2w5
5592,Union Oil Company of California,UNOCAL CORP,unocal,union oil company of california,org_COgAe5MtbLYwQVAiE2w5
7950,"The International Nickel Company, Inc.",INCO LTD,inco,international nickel,org_3O12UKd8pdv6Gj2KNctq
8111,Chicago Bridge & Iron Company,C B I INDUSTRIES INC,cbi industries,chicago bridge iron,org_Z3GvBv6FKSHtIl7CBaD5
9022,"Computer Associates Think, Inc.",C A INC,computer associates,computer associates think,org_NZwMhKGj9MK5x3VPdEQA
10798,"Computer Associates Think, Inc.",C A INC,computer associates,computer associates think,org_NZwMhKGj9MK5x3VPdEQA


In [253]:
s1 = set(final_gtm[~final_gtm['shortform'].isna()]['PERMNO'].unique())

s2 = set(final_gtm[final_gtm['shortform'].isna()]['PERMNO'].unique())

In [254]:
(s1 - s2)

{12092.0, 21020.0, 37161.0, 40484.0, 66325.0, 75298.0, 88939.0}

In [251]:
final_gtm.loc[954396,"finalmatch"] = False
final_gtm.loc[954396,"shortform"] = 'nan'

In [250]:
final_gtm[final_gtm['PERMNO']==88939.0]

,patent_id,organization,orglow9,PERMNO,PERMCO,COMNAM,comlow4,assignee_id,type,sequence,...,comlow_short,org_short,com_short,new_fr,new_ft,new_fpr,match,nmatch,finalmatch,shortform
170201,5272603,ITC Incorporated,itc,88939.0,7600.0,INDUSTRIAL TRAINING CORP,industrial training,org_Yz3DRxhIZCVE8cyw2HbK,2.0,0.0,...,it,ii,itc,18.0,18.0,33.0,NaN,True,True,short form
232673,4925140,ITC Incorporated,itc,88939.0,7600.0,INDUSTRIAL TRAINING CORP,industrial training,org_Yz3DRxhIZCVE8cyw2HbK,2.0,0.0,...,it,ii,itc,18.0,18.0,33.0,NaN,True,True,short form
346276,4987845,"ITC, Incorporated",itc,88939.0,7600.0,INDUSTRIAL TRAINING CORP,industrial training,org_Yz3DRxhIZCVE8cyw2HbK,2.0,0.0,...,it,ii,itc,18.0,18.0,33.0,NaN,True,True,short form
377257,5297010,ITC Incorporated,itc,88939.0,7600.0,INDUSTRIAL TRAINING CORP,industrial training,org_Yz3DRxhIZCVE8cyw2HbK,2.0,0.0,...,it,ii,itc,18.0,18.0,33.0,NaN,True,True,short form
858494,4738216,"ITC, Incorporated",itc,88939.0,7600.0,INDUSTRIAL TRAINING CORP,industrial training,org_Yz3DRxhIZCVE8cyw2HbK,2.0,0.0,...,it,ii,itc,18.0,18.0,33.0,NaN,True,True,short form
885326,4969073,"ITC, Incorporated",itc,88939.0,7600.0,INDUSTRIAL TRAINING CORP,industrial training,org_Yz3DRxhIZCVE8cyw2HbK,2.0,0.0,...,it,ii,itc,18.0,18.0,33.0,NaN,True,True,short form
1006798,5588908,"ITC, Incorporated",itc,88939.0,7600.0,INDUSTRIAL TRAINING CORP,industrial training,org_Yz3DRxhIZCVE8cyw2HbK,2.0,0.0,...,it,ii,itc,18.0,18.0,33.0,NaN,True,True,short form
1110401,5615631,ITC Incorporated,itc,88939.0,7600.0,INDUSTRIAL TRAINING CORP,industrial training,org_Yz3DRxhIZCVE8cyw2HbK,2.0,0.0,...,it,ii,itc,18.0,18.0,33.0,NaN,True,True,short form


In [165]:
final_gtm.PERMNO.nunique()

6728

In [166]:
tt.columns

Index(['organization', 'orglow9', 'PERMNO', 'PERMCO', 'COMNAM', 'comlow4',
       'temp_match', 'assignee_id', 'fuzz_ratio', 'fuzz_token', 'fuzz_fpr',
       'orglow_first', 'comlow_first', 'same', 'comlow_new', 'orglow_new',
       'orglow_short', 'comlow_short', 'org_short', 'com_short', 'new_fr',
       'new_ft', 'new_fpr', 'orgnew_first', 'comnew_first', 'same_new',
       'ofuzz_shortlow', 'cfuzz_shortlow', 'ofuzz_short', 'cfuzz_short',
       'match', 'oshrtlow_len', 'cshrtlow_len', 'oshrt_len', 'cshrt_len',
       'org_len', 'com_len', 'nmatch'],
      dtype='object')

In [167]:
tt.match.value_counts()

True    13114
Name: match, dtype: int64

In [168]:
tt.nmatch.value_counts()

False    16192
True     13720
Name: nmatch, dtype: int64

In [169]:
tt.temp_match.value_counts()

TRUE                           13129
shoud match                      464
FALSE                            410
should match on lang change       78
short form                        49
True                               2
False                              2
Name: temp_match, dtype: int64

In [170]:
tt1 = tt[['organization', 'orglow9', 'PERMNO', 'COMNAM', 'comlow4',
       'temp_match', 'fuzz_ratio', 'fuzz_token', 'fuzz_fpr',
         'comlow_new', 'orglow_new',
       'orglow_short', 'comlow_short', 'org_short', 'com_short', 'new_fr',
       'new_ft', 'new_fpr', 
       
       'match',  'nmatch']]

In [171]:
tt1.to_csv("tt1.csv", index = False)

In [549]:
w1 = set(final_gtm[(final_gtm['same']==False)&(final_gtm['same_new']==False)
             ]['PERMNO'].unique())

w2 = set(final_gtm[(final_gtm['same']==True)|(final_gtm['same_new']==True)
             ]['PERMNO'].unique())

In [555]:
w3 = list(w1 - w2)

In [556]:
w3

[69121.0,
 47108.0,
 41989.0,
 18438.0,
 48136.0,
 65032.0,
 57357.0,
 19984.0,
 89616.0,
 45081.0,
 37402.0,
 45604.0,
 51772.0,
 38974.0,
 78404.0,
 40521.0,
 40010.0,
 69199.0,
 30294.0,
 51799.0,
 42585.0,
 35422.0,
 81004.0,
 26235.0,
 85632.0,
 61576.0,
 66704.0,
 81042.0,
 22163.0,
 82581.0,
 62616.0,
 18585.0,
 80029.0,
 81060.0,
 41128.0,
 55467.0,
 44223.0,
 22728.0,
 88269.0,
 68304.0,
 82642.0,
 58579.0,
 49876.0,
 50905.0,
 88309.0,
 27385.0,
 65787.0,
 77058.0,
 43780.0,
 88846.0,
 77075.0,
 29971.0,
 38682.0,
 46877.0,
 26403.0,
 63782.0,
 90408.0,
 81193.0,
 81194.0,
 76080.0,
 46907.0,
 10558.0,
 47423.0,
 45890.0,
 57154.0,
 70981.0,
 53575.0,
 54114.0,
 85858.0,
 92012.0,
 40301.0,
 77165.0,
 43377.0,
 65402.0,
 23421.0,
 77182.0,
 19845.0,
 19852.0,
 62885.0,
 51115.0,
 25005.0,
 75186.0,
 47029.0,
 57795.0,
 89027.0,
 59336.0,
 85449.0,
 47562.0,
 49613.0,
 41937.0,
 93140.0,
 67540.0,
 52695.0,
 87009.0,
 18403.0,
 41444.0,
 26084.0,
 53735.0,
 48120.0,
 27131.0]

In [ ]:
10558.0

In [553]:
final_gtm[final_gtm['PERMNO']==10558.0][['organization','COMNAM']]

,organization,COMNAM
147106,"Proctor-Silex, Inc.",WEAREVER PROCTOR SILEX INC


In [558]:
final_gtm[final_gtm['PERMNO'].isin(w2)][['PERMNO','organization','COMNAM']].nunique()

PERMNO           6629
organization    30035
COMNAM           7269
dtype: int64

100, 299, 104

6629
347
(5%)
6052
(89.93%)
7269
531
6498
30035
351
17477


In [561]:
605200/7269

91.29582139085835

In [215]:
tt1 = pd.read_csv("tt1.csv")

In [217]:
tt2 = tt1.loc[tt1['temp_match']=="short form"][['organization','COMNAM','comlow4','orglow9', "temp_match"]]

In [218]:
tt2.columns

Index(['organization', 'COMNAM', 'comlow4', 'orglow9', 'temp_match'], dtype='object')

In [219]:
tt2.rename(columns={"temp_match":"shortform"}, inplace = True)

In [238]:
final_gtm = pd.merge(final_gt, tt2, left_on = ['organization','COMNAM','comlow4','orglow9'],
                    right_on = ['organization','COMNAM','comlow4','orglow9'], how = "left")

In [239]:
final_gtm['shortform'].value_counts()

short form    1837
Name: shortform, dtype: int64

In [329]:
final_gtm[final_gtm['shortform']=="short form"]['PERMNO'].nunique()

22

In [304]:
final_gt.same_new.value_counts()

True     234683
False     41976
Name: same_new, dtype: int64

In [394]:
final_gtm[(final_gtm['finalmatch']==True)]['PERMNO'].nunique()

6729

In [415]:
final_gtm.finalmatch.value_counts()

True     1289576
False         10
Name: finalmatch, dtype: int64

In [469]:
final_gtm = final_gtm[final_gtm['finalmatch']==True]

In [470]:
final_gtm['PERMNO'].nunique()

6729

In [509]:
final_gtm['organization'].nunique()

30314

In [255]:
final_gtm.to_csv('groundtruth_kpss_pv_crsp_28Apr20.csv', index = False)

In [472]:
final_gtm['orglow_first'] = final_gtm['orglow9'].apply(lambda x: x[0])
final_gtm['comlow_first'] = final_gtm['comlow4'].apply(lambda x: x[0])

final_gtm['same'] = final_gtm['orglow_first']==final_gtm['comlow_first']

In [473]:
final_gtm[(final_gtm['same']==False)&(final_gtm['same_new']==False)]['PERMNO'].nunique() ##361

352

In [474]:
final_gtm[(final_gtm['same']==False)&(final_gtm['same_new']==False)][['organization','COMNAM']]

,organization,COMNAM
30,E. I. du Pont de Nemours and Company,DU PONT E I DE NEMOURS & CO
38,A. E. Staley Manufacturing Company,STALEY A E MANUFACTURING CO
103,Shin Caterpillar Mitsubishi Ltd,CATERPILLAR INC
111,Telefonaktiebolaget LM Ericsson,ERICSSON L M TELEPHONE CO
122,Truth Div. of SPX Corporation,S P X CORP
...,...,...
1289401,E. I. du Pont de Nemours and Company,DU PONT E I DE NEMOURS & CO
1289461,E. I. Du Pont de Nemours and Company,DU PONT E I DE NEMOURS & CO
1289480,Telefonaktiebolaget L M Ericsson (publ),ERICSSON L M TELEPHONE CO
1289504,Eli Lilly and Company,LILLY ELI & CO
